In [1]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Model
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Load the GPT-2 tokenizer

from transformer import SimpleTransformer
from one_step_thoughtsformer import ThoughtsFormer
device = torch.device("cuda")
thoughtsformer = ThoughtsFormer(thought_length=0, vocab_size= 50257, max_sequence_length=1024, verbose=False, num_layers=2).to(device)
transformer = SimpleTransformer(vocab_size=50257, max_sequence_length=1024, num_layers=2).to(device)

embeds = torch.ones(1,32,768).to(device)
padding = torch.zeros(1,32).to(device)


In [20]:
thoughtsformer.forward_ppo(embeds,padding,0)[0].shape

c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\torch\nn\functional.py:5193: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(


torch.Size([1, 32, 50257])

In [1]:
from datasets import load_dataset
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset

class TinyShakespeareDataset(Dataset):
    def __init__(self, token_window_size, window_offset):
        self.token_window_size = token_window_size
        self.window_offset = window_offset
        
        tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        text = load_dataset("tiny_shakespeare", split="train")['text'][0]
        self.tokens = tokenizer.encode(text, return_tensors="pt")[0]

    def __len__(self):
        return (len(self.tokens) - self.token_window_size) // self.window_offset + 1

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self):
            raise IndexError("Index out of bounds")
        
        start = idx * self.window_offset
        end = start + self.token_window_size + 1
        
        return self.tokens[start:end-1], self.tokens[start+1:end]

In [2]:
from typing import Iterable, Tuple

token_window_size = 1024
window_offset = 256

dataset: Iterable[Tuple[torch.Tensor, torch.Tensor]]  = TinyShakespeareDataset(token_window_size,window_offset)


c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


In [18]:
for i in dataset:
    if i[1].shape[0] != 1024:
        print(i[0].shape[0])


In [55]:
import torch.nn.functional as F

thought_adam = torch.optim.Adam(params = thoughtsformer.parameters(), lr=0.001)
trans_adam = torch.optim.Adam(params = transformer.parameters(), lr=0.001)
loss_fn = F.cross_entropy

thoughtsformer.train()
for idx, (x, y) in enumerate(train_loader):
    x = x.to(device)
    y = y.to(device)
    
    mask = torch.zeros_like(x).to(device)
    
    # Training step
    thoughts_logits = thoughtsformer.forward_ppo_with_tokens(x, mask.bool(), 0)[0]
    thoughts_logits, y = thoughts_logits.squeeze(0), y.squeeze(0)
    thought_loss = loss_fn(thoughts_logits, y)
    

    trans_logits = transformer.forward_tokens(x, mask.bool())
    trans_logits = trans_logits.squeeze(0)
    trans_loss = loss_fn(trans_logits, y)
    
    thought_adam.zero_grad()
    thought_loss.backward()
    thought_adam.step()
    
    trans_adam.zero_grad()
    trans_loss.backward()
    trans_adam.step()
    
    print(f"thoughtsformer loss at step {idx}: {thought_loss.item()}")
    print(f"transformer loss at step {idx}: {trans_loss.item()}")
    
    # Evaluation step
    if idx % 10 == 0:
        thoughtsformer.eval()
        transformer.eval()
        
        with torch.no_grad():
            eval_losses = {'thoughtsformer': 0, 'transformer': 0}
            num_eval_batches = 0
            
            for eval_x, eval_y in test_loader:
                eval_x = eval_x.to(device)
                eval_y = eval_y.to(device)
                eval_mask = torch.zeros_like(eval_x).to(device)
                
                # Evaluate thoughtsformer
                eval_thoughts_logits = thoughtsformer.forward_ppo_with_tokens(eval_x, eval_mask.bool(), 0)[0]
                eval_thoughts_logits, eval_y = eval_thoughts_logits.squeeze(0), eval_y.squeeze(0)
                eval_thought_loss = loss_fn(eval_thoughts_logits, eval_y)
                eval_losses['thoughtsformer'] += eval_thought_loss.item()
                
                # Evaluate transformer
                eval_trans_logits = transformer.forward_tokens(eval_x, eval_mask.bool())
                eval_trans_logits = eval_trans_logits.squeeze(0)
                eval_trans_loss = loss_fn(eval_trans_logits, eval_y)
                eval_losses['transformer'] += eval_trans_loss.item()
                
                num_eval_batches += 1
                break
            
            avg_eval_losses = {k: v / num_eval_batches for k, v in eval_losses.items()}
            print(f"Evaluation at step {idx}:")
            print(f"  Avg thoughtsformer eval loss: {avg_eval_losses['thoughtsformer']:.4f}")
            print(f"  Avg transformer eval loss: {avg_eval_losses['transformer']:.4f}")
        
        thoughtsformer.train()
        transformer.train()
    break

thoughtsformer loss at step 0: 7.856191158294678
transformer loss at step 0: 9.520978927612305
Evaluation at step 0:
  Avg thoughtsformer eval loss: 10.3156
  Avg transformer eval loss: 8.4069


c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\torch\nn\modules\transformer.py:409: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


In [177]:
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer, GPT2Model
from datasets import load_dataset
import torch

batch_size = 24

token_count = 301966
token_window_size = 1000
window_offset = 500


class TinyShakespeareDataset(Dataset):
    def __init__(self, token_window_size, window_offset):
        self.token_window_size = token_window_size
        self.window_offset = window_offset
        
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        dataset = load_dataset("tiny_shakespeare", split="train", trust_remote_code=True)
        text = dataset[0]['text']  # It's small, just 1 million characters
        self.tokens = self.tokenizer.encode(text, return_tensors="pt")
        self.model = GPT2Model.from_pretrained("gpt2")
        
        self.cached_overlap = None
        self.cached_start = None  # Initialize cached_start
        
    def __len__(self):
        return (self.tokens.shape[1] - self.token_window_size) // self.window_offset + 1
    
    def __getitem__(self, idx):
        with torch.no_grad():
            start = idx * self.window_offset
            end = start + self.token_window_size

            # Check if we have cached overlap and can reuse it
            if self.cached_overlap is not None and self.cached_start == start:
                # Reuse the cached overlap (500-1000 in the previous iteration)
                cached_part = self.cached_overlap
                # Compute only the new part (1000-1500)
                new_part = self.tokens[0, end:end + self.window_offset]
                new_embeddings = self.model(new_part.unsqueeze(0)).last_hidden_state.squeeze(0)
                
                # Combine the cached part and the newly computed part
                total_embeddings = torch.cat((cached_part, new_embeddings), dim=0)
            else:
                # Compute the embeddings for the full window (0-1000)
                total_tokens = self.tokens[0, start:end]  # Corrected
                total_embeddings = self.model(total_tokens.unsqueeze(0)).last_hidden_state.squeeze(0)
                
            # Cache the overlap (500-1000 for the next iteration)
            self.cached_overlap = total_embeddings[-self.window_offset:]
            self.cached_start = start + self.window_offset  # Update cached_start

            # Prepare input (0-1000) and target (1-1001) tokens
            x = total_embeddings[:self.token_window_size, :]
            y = self.tokens[0, start + 1:end + 1]  # Target tokens (1-1001)


        return x, y
            
dataset = TinyShakespeareDataset(token_window_size, window_offset)

# Example to access an item and trigger print statements
# You can call this in a loop or at specific indices to see the output
x, y = dataset[0]  # Change the index to test other items


c:\Users\bicke\anaconda3\envs\megatron\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (301966 > 1024). Running this sequence through the model will result in indexing errors


In [191]:
class TinyShakespeareDataset2(Dataset):
    def __init__(self):
        output_embeddings_file = 'embeddings.pt'  # File name for the saved embeddings
        data = torch.load(output_embeddings_file)

        # Extract embeddings and targets
        self.embeddings = data['embeddings']
        self.targets = data['targets']

    def __len__(self):
        return self.embeddings.shape[0]
    
    def __getitem__(self, idx):
        return self.embeddings[idx,:,:], self.targets[idx,:]

In [192]:
for idx, (x, y) in enumerate(TinyShakespeareDataset2()):
    print(idx)

C:\Users\bicke\AppData\Local\Temp\ipykernel_22252\3375324087.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(output_embeddings_file)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [178]:
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [182]:
output_embeddings_file = 'embeddings.pt'  # Output file name for saving embeddings


# Preprocess and save embeddings
all_embeddings = []
all_targets = []

for idx, (x, y) in enumerate(TinyShakespeareDataset2):
    all_embeddings.append(x)
    all_targets.append(y)
    print(idx, end = " | ")
    if idx % 50 == 0:
        print()
        all_embeddings2 = torch.cat(all_embeddings, dim=0)  # Shape: (N, token_window_size, embedding_dim)
        all_targets2 = torch.cat(all_targets, dim=0)        # Shape: (N, token_window_size)

        # Save the embeddings and targets
        torch.save({'embeddings': all_embeddings2, 'targets': all_targets2}, output_embeddings_file)


# Concatenate all embeddings and targets
all_embeddings = torch.cat(all_embeddings, dim=0)  # Shape: (N, token_window_size, embedding_dim)
all_targets = torch.cat(all_targets, dim=0)        # Shape: (N, token_window_size)

# Save the embeddings and targets
torch.save({'embeddings': all_embeddings, 'targets': all_targets}, output_embeddings_file)


    

0 | 
1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 | 29 | 30 | 31 | 32 | 33 | 34 | 35 | 36 | 37 | 38 | 39 | 40 | 41 | 42 | 43 | 44 | 45 | 46 | 47 | 48 | 49 | 50 | 
51 | 52 | 53 | 54 | 55 | 56 | 57 | 58 | 59 | 60 | 

In [186]:
all_embeddings.shape

torch.Size([481, 1000, 768])

In [183]:
output_embeddings_file = 'embeddings.pt'  # File name for the saved embeddings
data = torch.load(output_embeddings_file)

# Extract embeddings and targets
embeddings = data['embeddings']
targets = data['targets']


C:\Users\bicke\AppData\Local\Temp\ipykernel_22252\1003614756.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(output_embeddings_file)


In [187]:
print(targets.shape)

torch.Size([481, 1000])


In [57]:
tokenizer.decode(tokens[0])

'e'

In [34]:
text[0:13]

'First Citizen'

In [36]:
tokens[0]

tensor([ 5962, 22307])